In [14]:
from rdflib import Graph
from cmatcher.module_search.pagerank import gen_pagerank_sparql_queries
from transformers import AutoTokenizer, AutoModel, BitsAndBytesConfig, AutoModelForCausalLM
import torch

from cmatcher.rag.rag_reduce import ont_query_reduce

In [12]:
tokenizer = AutoTokenizer.from_pretrained('Salesforce/SFR-Embedding-2_R')
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)
model = AutoModel.from_pretrained(
    'Salesforce/SFR-Embedding-2_R',
    quantization_config=quantization_config,
    device_map='auto',
)
model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

MistralModel(
  (embed_tokens): Embedding(32000, 4096, padding_idx=2)
  (layers): ModuleList(
    (0-31): 32 x MistralDecoderLayer(
      (self_attn): MistralSdpaAttention(
        (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        (rotary_emb): MistralRotaryEmbedding()
      )
      (mlp): MistralMLP(
        (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
        (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
        (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): MistralRMSNorm()
      (post_attention_layernorm): MistralRMSNorm()
    )
  )
  (norm): MistralRMSNorm()
)

In [6]:
base_path = '/projets/melodi/gsantoss/data/complex/conference/ont/'
o1 = Graph().parse(base_path + 'cmt.owl')
o2 = Graph().parse(base_path + 'conference.owl')

In [10]:

queries1 = gen_pagerank_sparql_queries(o1)

In [13]:
prompt = 'Given the following SPARQL query, retrieve relevant entities that are related to the query'

for query in queries1:
    
    module1 = ont_query_reduce(model, tokenizer, run[1], o1, query, prompt, max_entities=10, batch_size=2)
    module2 = ont_query_reduce(model, tokenizer, run[2], o2, query, prompt, max_entities=10, batch_size=2)
    
    
    break

SELECT DISTINCT ?x WHERE {?x a <http://cmt#Conference>.}
